## Using Nvidia GPU with less than 12Gig VRAM

In [1]:
# To keep track of GPU usage in real time.
# uncomment and run following command in terminal (ubuntu):
# watch -n nvidia-smi

### Imports & Setup
We start by importing all the necessary libraries and modules. 

In this cell, we bring in tools from the smolagents package, the Transformers library, and yfinance, along with utilities for managing dates and timezones. 

This setup establishes the environment we need to build our local agent that can execute code, perform web searches, and interact with financial data.

In [1]:
from typing import Any, Optional
from smolagents.tools import Tool
from smolagents import CodeAgent, TransformersModel, tool, DuckDuckGoSearchTool
from transformers import AutoTokenizer
import yaml

import yfinance as yf
from datetime import datetime
import pytz


In [ ]:
# Setting up the predefined prompts
with open("prompts.yaml", 'r') as stream:
    prompt_templates = yaml.safe_load(stream)

### Market Status Function

Here, we define a function that checks the current market status for a given ticker, and we decorate it as a tool. 

Using the `yfinance` library, the function retrieves stock information and then compares the current time in New York (`EST Time`) against standard trading hours. 

We also handle scenarios like weekends and any potential errors, ensuring that our agent can reliably tell whether the market is open or closed.

In [4]:
# first tool
@tool
def check_NA_market_status(ticker: str) -> str:
    """
    A tool that checks if the stock market is open for the given ticker and returns the current price if open,
    or the previous close price if the market is closed.
    Args:
        ticker: The stock ticker symbol (e.g., 'AAPL', 'GOOG').
    Returns:
        A message indicating whether the market is open or closed, 
        and the price of the stock at the respective time.
    """

    # Get the current time in Eastern Time Zone
    eastern = pytz.timezone('US/Eastern')
    now = datetime.now(eastern)
    current_time = now.time()
    current_day = now.weekday()  # Monday is 0 and Sunday is 6

    market_open_time = datetime.strptime("09:30:00", "%H:%M:%S").time()
    market_close_time = datetime.strptime("16:00:00", "%H:%M:%S").time()

    if current_day < 5 and market_open_time <= current_time <= market_close_time:
        stock = yf.Ticker(ticker)
        # Request intraday data: 1-day period with 1-minute interval
        current_price = stock.history(period="1d", interval="1m")['Close'].iloc[-1]
        return f"Market is open. Current price of {ticker}: ${current_price:.2f}"
    else:
        stock = yf.Ticker(ticker)
        previous_close = stock.history(period="1d")['Close'].iloc[-1]
        return f"Market is closed. Previous close price of {ticker}: ${previous_close:.2f}"



In [5]:
# second tool
@tool
def fibonacci_tool(n: int) -> int:
    """
    A simple tool that returns the nth Fibonacci number.

    Args:
        n: The index of the Fibonacci sequence to retrieve (int)

    Returns:
        int: The nth Fibonacci number.
    """
    if n <= 0:
        return 0
    elif n == 1:
        return 1
    else:
        a, b = 0, 1
        for _ in range(2, n + 1):
            a, b = b, a + b
        return b

### Bypassing the LLM's training date cutoff

In the following snippet, we create a tool that leverages `DuckDuckGoSearch` to fetch up-to-date information from the web. 

We designed this function to perform a search based on a user’s query and return the results or an error message if something goes wrong.

This tool empowers our agent with the ability to quickly retrieve relevant information on demand. Since models training have cutoff dates, this tool will allow us to bypass this cutoff limitation by researching the web for up to date answers to any query we may have.

In [6]:
# third tool
@tool
def duckduckgo_search(query: str) -> str:
    """
    A tool that performs a real-time DuckDuckGo search for the given query and returns the top results.
    
    Args:
        query: The search query string (e.g., "Latest AI research 2025").
        
    Returns:
        A string containing the search results. If the search tool returns a formatted string,
        it is returned directly; otherwise, if it returns a list of dictionaries with 'title' and 'url'
        keys, the results are formatted for readability.
    """
    search_tool = DuckDuckGoSearchTool()
    results = search_tool(query)
    
    if isinstance(results, str):
        return results
    
    formatted_results = "\n".join(
        [f"{item.get('title', 'No Title')} -> {item.get('url', 'No URL')}" for item in results]
    )
    return formatted_results

In [7]:
# final answer class tool used after reasoning
class FinalAnswerTool(Tool):
    name = "final_answer"
    description = "Provides a final answer to the given problem."
    inputs = {'answer': {'type': 'any', 'description': 'The final answer to the problem'}}
    output_type = "any"

    def forward(self, answer: Any) -> Any:
        return answer

    def __init__(self, *args, **kwargs):
        self.is_initialized = False

# instanciate tool
final_answer = FinalAnswerTool()

### Initializing our AI agent.

To instanciate our agent, we must first create a model instance by setting up a specific Transformers model and its tokenizer (with CUDA enabled and controlled token limits). 

In [ ]:
# pick a model and instanciate it
# model_name= "HuggingFaceTB/SmolLM2-360M-Instruct"
model_name="HuggingFaceTB/SmolLM2-1.7B-Instruct"
# model_name = "HuggingFaceTB/SmolLM-135M-Instruct"
model = TransformersModel(model_id=model_name, device_map="auto", max_new_tokens=200)

Then, we instantiate our `CodeAgent`, integrating the market status, fibonacci and web search tools we defined earlier.

 We also provide a system message to guide the agent’s behavior, so it knows how to use these tools to effectively help users with their queries

In [8]:
# finaly, create the agent and pass the model, tools, number of steps it should reflect for, prompt template...ect

agent = CodeAgent(
    model=model,
    tools=[final_answer,fibonacci_tool, check_NA_market_status, duckduckgo_search], ## add your tools here (don't remove final answer)
    max_steps=6,
    verbosity_level=1,
    grammar=None,
    planning_interval=None,
    name=None,
    description=None,
    prompt_templates=prompt_templates
)

Finally, we query our `agent` by passing in our queries to the `.run()` method.

In [10]:
print("CodeAgent:", agent.run("Could you give me the 118th number in the Fibonacci sequence?"))

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Could you give me the 118th number in the Fibonacci sequence?                                                   │
│                                                                                                                 │
╰─ TransformersModel - HuggingFaceTB/SmolLM2-1.7B-Instruct ───────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  answer = fibonacci_tool(n=118)                                                                                   
  final_answer(answer)                                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 2046711111473984623691759

[Step 0: Duration 4.45 seconds| Input tokens: 2,511 | Output tokens: 63]

CodeAgent: 2046711111473984623691759


In [11]:
print("CodeAgent:", agent.run("Could you give me the current market price of the stock VEQT.TO?"))

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Could you give me the current market price of the stock VEQT.TO?                                                │
│                                                                                                                 │
╰─ TransformersModel - HuggingFaceTB/SmolLM2-1.7B-Instruct ───────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  ticker = "VEQT.TO"                                                                                               
  current_price = check_NA_market_status(ticker=ticker)                                                            
  final_answer(current_price)                                                                                      
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Market is open. Current price of VEQT.TO: $46.64

[Step 0: Duration 6.54 seconds| Input tokens: 2,512 | Output tokens: 102]

CodeAgent: Market is open. Current price of VEQT.TO: $46.64


In [9]:
print("CodeAgent:", agent.run("Could you search the web for the latest AI research in 2025"))

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Could you search the web for the latest AI research in 2025                                                     │
│                                                                                                                 │
╰─ TransformersModel - HuggingFaceTB/SmolLM2-1.7B-Instruct ───────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  query = "Latest AI research 2025"                                                                                
  answer = duckduckgo_search(query=query)                                                                          
  final_answer(answer)                                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: ## Search Results

[The 10 Biggest AI Trends Of 2025 Everyone Must Be Ready For Today - 
Forbes](https://www.forbes.com/sites/bernardmarr/2024/09/24/the-10-biggest-ai-trends-of-2025-everyone-must-be-ready
-for-today/)
Discover the 10 major AI trends set to reshape 2025: from augmented working and real-time decision-making to 
advanced AI legislation and sustainable AI initiatives.

[AI in the workplace: A report for 2025 | McKinsey - McKinsey & 
Company](https://www.mckinsey.com/capabilities/mckinsey-digital/our-insights/superagency-in-the-workplace-empowerin
g-people-to-unlock-ais-full-potential-at-work)
Artificial intelligence has arrived in the workplace and has the potential to be as transformative as the steam 
engine was to the 19th-century Industrial Revolution. 1 "Gen AI: A cognitive industrial revolution," McKinsey, June
7, 2024. With powerful and capable large language models (LLMs) developed by Anthropic, Cohere, Google, Meta, 
Mistral, OpenAI, and others, we have entered a new ...

[Artificial Intelligence | Stanford Emerging Technology 
Review](https://setr.stanford.edu/technology/artificial-intelligence/2025)
• Artificial intelligence (AI) is a foundational technology that is supercharging other scientific fields and, like
electricity and the internet, has the potential to transform societies, economies, and politics worldwide. ... 
Research on foundational AI technologies is difficult to regulate, even among likeminded nations. It is even more 
...

[What's next for AI in 2025 - MIT Technology 
Review](https://www.technologyreview.com/2025/01/08/1109188/whats-next-for-ai-in-2025/)
AI model makers are also keen to pitch their generative products as research tools for scientists. OpenAI let 
scientists test its latest o1 model and see how it might support them in research. The ...

[Top 15 AI Trends for 2025: Expert Predictions You Need to 
Know](https://techstartups.com/2025/01/01/top-15-ai-trends-for-2025-expert-predictions-you-need-to-know/)
3. Open-Source AI Gains Momentum Last week, we wrote about how Deepseek outperformed OpenAI and Meta's latest 
models at a fraction of the cost.Deepseek, a free open-source AI model developed by a Chinese tech startup, 
exemplifies a growing trend in open-source AI, where accessible tools are pushing the boundaries of performance and
affordability.

[Five Trends in AI and Data Science for 2025 - MIT Sloan Management 
Review](https://sloanreview.mit.edu/article/five-trends-in-ai-and-data-science-for-2025/)
But we will incorporate the latest research whenever possible. Randy has just completed his annual survey of data, 
analytics, and AI executives, the 2025 AI & Data Leadership Executive Benchmark Survey, conducted by his 
educational firm, Data & AI Leadership Exchange; and Tom has worked on several surveys on generative AI and data, 
technology ...

[Artificial Intelligence News -- 
ScienceDaily](https://www.sciencedaily.com/news/computers_math/artificial_intelligence/)
Jan. 15, 2025 — A new initiative is challenging the conversation around the direction of artificial intelligence 
(AI). It charges that the current trajectory is inherently biased against non ...

[AI Advancements in 2025: The Next Big Innovations to 
Watch](https://techresearchs.com/artificial-intelligence/ai-advancements-in-2025-the-next-big-innovations-to-watch/
)
Artificial Intelligence (AI) continues to evolve, reshaping industries and daily life. In 2025, AI advancements 
will bring groundbreaking innovations, enhancing automation, decision-making, and problem-solving across various 
sectors. This article explores the latest AI advancements in 2025, their applications, and their transformative 
impact on businesses and society.

[10 Artificial Intelligence Trends in 2025 To Stay 
Ahead](https://northwest.education/insights/artificial-intelligence/10-top-artificial-intelligence-trends-to-stay-a
head/)
10 AI trends that will define 2025. The following trends highlight how artificial

[Step 0: Duration 6.77 seconds| Input tokens: 2,510 | Output tokens: 82]

CodeAgent: ## Search Results

[The 10 Biggest AI Trends Of 2025 Everyone Must Be Ready For Today - Forbes](https://www.forbes.com/sites/bernardmarr/2024/09/24/the-10-biggest-ai-trends-of-2025-everyone-must-be-ready-for-today/)
Discover the 10 major AI trends set to reshape 2025: from augmented working and real-time decision-making to advanced AI legislation and sustainable AI initiatives.

[AI in the workplace: A report for 2025 | McKinsey - McKinsey & Company](https://www.mckinsey.com/capabilities/mckinsey-digital/our-insights/superagency-in-the-workplace-empowering-people-to-unlock-ais-full-potential-at-work)
Artificial intelligence has arrived in the workplace and has the potential to be as transformative as the steam engine was to the 19th-century Industrial Revolution. 1 "Gen AI: A cognitive industrial revolution," McKinsey, June 7, 2024. With powerful and capable large language models (LLMs) developed by Anthropic, Cohere, Google, Meta, Mistral, OpenAI, and others, we have entere